In [391]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os  # permet d'aller écrire dans les répertoires
from openpyxl import Workbook
import pandas as pd

In [392]:
option = webdriver.ChromeOptions()
option.add_argument("--incognito")  # Navigation privée
# option.add_argument("kiosk")  # grand ecran sur mac
#prefs={"profile.managed_default_content_settings.images": 2, 'disk-cache-size': 4096 }
#option.add_experimental_option('prefs', prefs)

In [393]:
# Pour Mac
browser = webdriver.Chrome(
   executable_path="/Users/thomasbouiniere/Desktop/Scrapping/chromedriver", options=option)

# Pour windows
# browser = webdriver.Chrome(
#     executable_path=r"C:/Users/HP/OneDrive/Bureau/scrap/chromedriver.exe", options=option)

<ipython-input-393-41f305e4f926>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(


In [394]:
# ---------------------------------------------------------------
# Betclic
# ---------------------------------------------------------------
compets_betclic = {'ldc': 'https://www.betclic.fr/football-s1/ligue-des-champions-c8',
                   'pl': 'https://www.betclic.fr/football-s1/angl-premier-league-c3',
                   'l1': 'https://www.betclic.fr/football-s1/ligue-1-uber-eats-c4',
                   'liga': 'https://www.betclic.fr/football-s1/espagne-liga-primera-c7'}

res_betclic = {'ldc': {'cotes': [], 'noms': [], 'dates': []},
               'pl': {'cotes': [], 'noms': [], 'dates': []},
               'l1': {'cotes': [], 'noms': [], 'dates': []},
               'liga': {'cotes': [], 'noms': [], 'dates': []}}

In [395]:
for i, compet in enumerate(compets_betclic):
    browser.get(compets_betclic[compet])
    time.sleep(1)
    if i ==1:
        time.sleep(2)
        button = browser.find_element_by_xpath(
            '//*[@id="popin_tc_privacy_button_2"]')
        button.click()

    cotes_temp = browser.find_elements_by_xpath(
        '/html/body/app-desktop/div[1]/div/bcdk-content-scroller/div/sports-competition/div/sports-events-list/bcdk-vertical-scroller/div/div[2]/div/div/div/div[2]/div/sports-events-event/a/sports-events-event-markets/sports-markets-default[1]/div/sports-selections-selection/div[1]/span[2]')
    for x in cotes_temp:
        res_betclic[compet]['cotes'].append(x.text)


    nom_temp = browser.find_elements_by_xpath(
        '/html/body/app-desktop/div[1]/div/bcdk-content-scroller/div/sports-competition/div/sports-events-list/bcdk-vertical-scroller/div/div[2]/div/div/div/div[2]/div/sports-events-event/a/div/scoreboards-scoreboard/scoreboards-scoreboard-global/div/div/div')
    for x in nom_temp:
        res_betclic[compet]['noms'].append(x.text)

    date_temp = browser.find_elements_by_xpath(
        '/html/body/app-desktop/div[1]/div/bcdk-content-scroller/div/sports-competition/div/sports-events-list/bcdk-vertical-scroller/div/div[2]/div/div/div/div[2]/div/sports-events-event/a/div/sports-events-event-info/div')
    for x in date_temp:
        res_betclic[compet]['dates'].append(x.text)

<ipython-input-395-b413b2cab6c6>:10: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  cotes_temp = browser.find_elements_by_xpath(
<ipython-input-395-b413b2cab6c6>:16: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  nom_temp = browser.find_elements_by_xpath(
<ipython-input-395-b413b2cab6c6>:21: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  date_temp = browser.find_elements_by_xpath(
<ipython-input-395-b413b2cab6c6>:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = browser.find_element_by_xpath(


In [396]:
for compet in res_betclic:
    for i,name in enumerate(res_betclic[compet]['noms']):
        if name =='Barcelone':
            res_betclic[compet]['noms'][i] = 'FC Barcelone'
        elif name == 'Séville':
            res_betclic[compet]['noms'][i] = 'FC Séville'

    res_betclic[compet]['cotes']= [[res_betclic[compet]['cotes'][i],res_betclic[compet]['cotes'][i+1],res_betclic[compet]['cotes'][i+2]] for i in range(len(res_betclic[compet]['cotes'])-2) if i%3==0]
    res_betclic[compet]['noms']=[[res_betclic[compet]['noms'][i],res_betclic[compet]['noms'][i+1]] for i in range(len(res_betclic[compet]['noms'])-1) if i%2 ==0]
print(res_betclic['ldc']['noms'])



[['Manchester City', 'Atletico Madrid'], ['Benfica', 'Liverpool'], ['Villarreal', 'Bayern Munich'], ['Chelsea', 'Real Madrid']]


In [397]:
print(res_betclic)


{'ldc': {'cotes': [['1,36', '4,40', '6,75'], ['7,70', '4,30', '1,34'], ['4,95', '4,00', '1,53'], ['', '', '']], 'noms': [['Manchester City', 'Atletico Madrid'], ['Benfica', 'Liverpool'], ['Villarreal', 'Bayern Munich'], ['Chelsea', 'Real Madrid']], 'dates': ['05/04/2022 21:00', '05/04/2022 21:00', '06/04/2022 21:00', '06/04/2022 21:00']}, 'pl': {'cotes': [['1,11', '8,50', '17'], ['11,50', '6,00', '1,20'], ['2,30', '3,45', '2,75'], ['1,33', '4,65', '8,15'], ['2,78', '3,00', '2,52'], ['1,56', '3,70', '5,70'], ['1,50', '4,25', '5,25'], ['1,73', '3,60', '4,25'], ['1,48', '4,15', '5,85'], ['3,80', '3,25', '1,93'], ['2,55', '3,10', '2,65'], ['2,38', '3,00', '2,82'], ['4,00', '3,55', '1,72'], ['2,65', '3,15', '2,40'], ['4,60', '3,65', '1,62'], ['1,56', '3,65', '5,15'], ['2,85', '3,15', '2,25'], ['2,65', '3,05', '2,48'], ['2,07', '3,25', '3,15'], ['3,05', '3,20', '2,13']], 'noms': [['Liverpool', 'Watford'], ['Burnley', 'Manchester City'], ['Leeds', 'Southampton'], ['Chelsea', 'Brentford'], ['W

In [398]:
# ---------------------------------------------------------------
# Winamax
# ---------------------------------------------------------------

compets_winamax = {'ldc': 'https://www.winamax.fr/paris-sportifs/sports/1/800000542/23',
                   'pl': 'https://www.winamax.fr/paris-sportifs/sports/1/1/1',
                   'l1': 'https://www.winamax.fr/paris-sportifs/sports/1/7/4',
                   'liga': 'https://www.winamax.fr/paris-sportifs/sports/1/32/36'}

res_winamax = {'ldc': {'cotes': [], 'noms': [], 'dates': []},
               'pl': {'cotes': [], 'noms': [], 'dates': []},
               'l1': {'cotes': [], 'noms': [], 'dates': []},
               'liga': {'cotes': [], 'noms': [], 'dates': []}}

In [399]:
for i, compet in enumerate(compets_winamax):
    browser.get(compets_winamax[compet])
    time.sleep(1)

    '''On récupère les cotes'''

    cotes_liga_temp = browser.find_elements_by_xpath(
        '//*[@id="app"]/div[1]/div[1]/div/div[2]/div/section/div[3]/div[1]/div/div/div/div/div[1]/a')
    cotes_liga = []
    try:
        for lien in cotes_liga_temp:
            for i in range(0, 3):
                x = lien.get_property('lastChild')
                x = x.get_property('lastChild')
                x = x.get_property('lastChild')
                x = x.get_property('children')[0]
                x = x.get_property('children')[i]
                x = x.get_property('children')[1]
                x = x.get_property('children')[0]
                x = x.get_property('children')[1]
                res_winamax[compet]['cotes'].append(x.text)
    except:
        None

    '''On récupère les noms d'équipes'''
    noms_temp = browser.find_elements_by_xpath(
        '//*[@id="app"]/div/div[1]/div/div[2]/div/section/div[3]/div[1]/div/div/div/div/div[1]/a/div[2]/div[1]/div[1]/div[1]/div/span')
    for x in noms_temp:
        res_winamax[compet]['noms'].append(x.text)

    '''On récupère les dates des matchs'''

    dates_temp = browser.find_elements_by_xpath(
        '//*[@id="app"]/div/div[1]/div/div[2]/div/section/div[3]/div[1]/div/div/div/div/div[1]/a/div[1]/div[2]/div')
    dates_temp = dates_temp[:-1]

    for x in dates_temp:
        res_winamax[compet]['dates'].append(x.text)

<ipython-input-399-64ccc6ec26ff>:7: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  cotes_liga_temp = browser.find_elements_by_xpath(
<ipython-input-399-64ccc6ec26ff>:26: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  noms_temp = browser.find_elements_by_xpath(
<ipython-input-399-64ccc6ec26ff>:33: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  dates_temp = browser.find_elements_by_xpath(


In [400]:
for compet in res_winamax: 
    while ' - ' in res_winamax[compet]['noms']:
        res_winamax[compet]['noms'].remove(' - ')
        
for compet in res_winamax:
    for i,name in enumerate(res_winamax[compet]['noms']):
        if name == 'CF Cadiz':
            res_winamax[compet]['noms'][i] = 'Cadix'


for compet in res_betclic:
    res_winamax[compet]['cotes']= [[res_winamax[compet]['cotes'][i],res_winamax[compet]['cotes'][i+1],res_winamax[compet]['cotes'][i+2]] for i in range(len(res_winamax[compet]['cotes'])-2) if i%3==0]
    res_winamax[compet]['noms']=[[res_winamax[compet]['noms'][i],res_winamax[compet]['noms'][i+1]] for i in range(len(res_winamax[compet]['noms'])-1) if i%2 ==0]

print(res_winamax)

{'ldc': {'cotes': [['1,40', '4,75', '8,50'], ['8,50', '4,80', '1,37'], ['2,05', '3,40', '3,75'], ['5,75', '4,25', '1,55']], 'noms': [['Manchester City', 'Atletico Madrid'], ['Benfica', 'Liverpool'], ['Chelsea', 'Real Madrid'], ['Villarreal', 'Bayern Munich']], 'dates': ['5 AVR. 2022 À 21:00', '5 AVR. 2022 À 21:00', '6 AVR. 2022 À 21:00', '6 AVR. 2022 À 21:00']}, 'pl': {'cotes': [['1,11', '9,50', '21'], ['12,50', '6,50', '1,21'], ['1,33', '5,00', '10'], ['2,85', '3,10', '2,60'], ['1,58', '3,85', '6,00'], ['2,35', '3,55', '2,85'], ['1,54', '4,40', '5,50'], ['1,78', '3,75', '4,40'], ['1,50', '4,30', '6,00'], ['3,95', '3,35', '1,98'], ['2,55', '3,15', '2,70'], ['2,45', '3,05', '2,90'], ['4,20', '3,70', '1,76'], ['1,58', '3,80', '5,40'], ['4,80', '3,80', '1,66'], ['2,75', '3,25', '2,45'], ['2,95', '3,25', '2,30'], ['3,15', '3,30', '2,20'], ['2,70', '3,15', '2,55'], ['2,10', '3,35', '3,25'], ['2,05', '3,50', '3,20']], 'noms': [['Liverpool', 'Watford'], ['Burnley', 'Manchester City'], ['Chels

In [401]:
# ---------------------------------------------------------------
# unibet
# ---------------------------------------------------------------

compets_unibet = {'ldc': 'https://www.unibet.fr/sport/football/ligue-des-champions/ligue-des-champions-matchs',
                  'pl': 'https://www.unibet.fr/sport/football/premier-league/matchs',
                  'l1': 'https://www.unibet.fr/sport/football/ligue-1-ubereats/ligue-1-matchs',
                  'liga': 'https://www.unibet.fr/sport/football/liga/matchs'}

res_unibet = {'ldc': {'cotes': [], 'noms': [], 'dates': []},
              'pl': {'cotes': [], 'noms': [], 'dates': []},
              'l1': {'cotes': [], 'noms': [], 'dates': []},
              'liga': {'cotes': [], 'noms': [], 'dates': []}}

In [402]:
for i, compet in enumerate(compets_unibet):
    browser.get(compets_unibet[compet])
    time.sleep(2)
    '''On cherche les côtes '''
    cotes_temp = browser.find_elements_by_xpath(
        '//*[@id="page__competitionview"]/div[1]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[2]')
    for match in cotes_temp:
        for i in range(0, 3):
            x = match.get_property('children')[0]
            x = x.get_property('children')[0]
            x = x.get_property('children')[0]
            x = x.get_property('children')[0]
            x = x.get_property('children')[i]
            x = x.get_property('children')[0]
            x = x.get_property('children')[2]
            res_unibet[compet]['cotes'].append(x.text)

    '''On cherche les noms d'équipes'''

    noms_temp = browser.find_elements_by_xpath(
        '//*[@id="page__competitionview"]/div[1]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div/div/div[2]/div[1]/div/span')
    for x in noms_temp:
        res_unibet[compet]['noms'].append(x.text.split('-'))

<ipython-input-402-e5da6e225fe2>:5: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  cotes_temp = browser.find_elements_by_xpath(
<ipython-input-402-e5da6e225fe2>:20: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  noms_temp = browser.find_elements_by_xpath(


In [403]:
for compet in res_unibet:
    res_unibet[compet]['cotes']= [[res_unibet[compet]['cotes'][i],res_unibet[compet]['cotes'][i+1],res_unibet[compet]['cotes'][i+2]] for i in range(len(res_unibet[compet]['cotes'])-2) if i%3==0]
for compet in res_unibet:
    for x in res_unibet[compet]['noms']:
        x[1]=x[1][1:]



print(res_unibet)

{'ldc': {'cotes': [['7.80', '4.35', '1.35'], ['1.37', '4.45', '6.85'], ['2.05', '3.28', '3.25'], ['5.00', '4.05', '1.53']], 'noms': [['Benfica ', 'Liverpool'], ['Manchester City ', 'Atletico Madrid'], ['Chelsea ', 'Real Madrid'], ['Villarreal ', 'Bayern Munich']], 'dates': []}, 'pl': {'cotes': [['1.08', '7.35', '15.20'], ['1.51', '3.58', '5.50'], ['10.80', '5.70', '1.17'], ['1.29', '4.45', '7.85'], ['2.20', '3.35', '2.68'], ['2.72', '2.95', '2.41'], ['1.46', '4.10', '5.10'], ['1.68', '3.48', '4.15'], ['1.43', '4.00', '5.65'], ['3.70', '3.18', '1.86'], ['2.43', '3.05', '2.60'], ['2.31', '2.95', '2.82'], ['4.00', '3.55', '1.68'], ['1.53', '3.65', '5.15'], ['4.60', '3.62', '1.59'], ['2.65', '3.12', '2.35'], ['2.85', '3.15', '2.20'], ['3.02', '3.18', '2.09'], ['2.02', '3.25', '3.12'], ['2.62', '3.05', '2.41'], ['1.98', '3.38', '3.10']], 'noms': [['Liverpool ', 'Watford'], ['Brighton ', 'Norwich City'], ['Burnley ', 'Manchester City'], ['Chelsea ', 'Brentford'], ['Leeds United ', 'Southampt

In [404]:
for compet in res_unibet:
    for matchs in res_unibet[compet]['noms']:
        if "tienne " in matchs:
            matchs.remove('tienne ')
        if "Etienne" in matchs:
            matchs.remove('Etienne')
        if " Marseille" in matchs:
            matchs[1] = "Marseille"

In [405]:
res = {
    'ldc' : {},
    'pl' : {},
    'l1' : {},
    'liga'  : {}
    }
sites = {'betclic': res_betclic,'winamax': res_winamax, 'unibet':res_unibet}
for site in sites:
    for comp in sites[site]:
        for i,match in enumerate(sites[site][comp]['noms']):
            if match[0][:2]+match[1][:2] not in res[comp]:
                res[comp][match[0][:2]+match[1][:2]] = {}
                res[comp][match[0][:2]+match[1][:2]]['equip1'] = match[0]
                res[comp][match[0][:2]+match[1][:2]]['equip2'] = match[1]
            res[comp][match[0][:2]+match[1][:2]][site] = sites[site][comp]['cotes'][i]

print(res)



{'ldc': {'MaAt': {'equip1': 'Manchester City', 'equip2': 'Atletico Madrid', 'betclic': ['1,36', '4,40', '6,75'], 'winamax': ['1,40', '4,75', '8,50'], 'unibet': ['1.37', '4.45', '6.85']}, 'BeLi': {'equip1': 'Benfica', 'equip2': 'Liverpool', 'betclic': ['7,70', '4,30', '1,34'], 'winamax': ['8,50', '4,80', '1,37'], 'unibet': ['7.80', '4.35', '1.35']}, 'ViBa': {'equip1': 'Villarreal', 'equip2': 'Bayern Munich', 'betclic': ['4,95', '4,00', '1,53'], 'winamax': ['5,75', '4,25', '1,55'], 'unibet': ['5.00', '4.05', '1.53']}, 'ChRe': {'equip1': 'Chelsea', 'equip2': 'Real Madrid', 'betclic': ['', '', ''], 'winamax': ['2,05', '3,40', '3,75'], 'unibet': ['2.05', '3.28', '3.25']}}, 'pl': {'LiWa': {'equip1': 'Liverpool', 'equip2': 'Watford', 'betclic': ['1,11', '8,50', '17'], 'winamax': ['1,11', '9,50', '21'], 'unibet': ['1.08', '7.35', '15.20']}, 'BuMa': {'equip1': 'Burnley', 'equip2': 'Manchester City', 'betclic': ['11,50', '6,00', '1,20'], 'winamax': ['12,50', '6,50', '1,21'], 'unibet': ['10.80', 

In [406]:
for compet in res:
    print(compet)
    print(len(res[compet]))
    print(len(res_betclic[compet]['noms']))
    print(len(res_winamax[compet]['noms']))
    print(len(res_unibet[compet]['noms']))
    print(res[compet])

ldc
4
4
4
4
{'MaAt': {'equip1': 'Manchester City', 'equip2': 'Atletico Madrid', 'betclic': ['1,36', '4,40', '6,75'], 'winamax': ['1,40', '4,75', '8,50'], 'unibet': ['1.37', '4.45', '6.85']}, 'BeLi': {'equip1': 'Benfica', 'equip2': 'Liverpool', 'betclic': ['7,70', '4,30', '1,34'], 'winamax': ['8,50', '4,80', '1,37'], 'unibet': ['7.80', '4.35', '1.35']}, 'ViBa': {'equip1': 'Villarreal', 'equip2': 'Bayern Munich', 'betclic': ['4,95', '4,00', '1,53'], 'winamax': ['5,75', '4,25', '1,55'], 'unibet': ['5.00', '4.05', '1.53']}, 'ChRe': {'equip1': 'Chelsea', 'equip2': 'Real Madrid', 'betclic': ['', '', ''], 'winamax': ['2,05', '3,40', '3,75'], 'unibet': ['2.05', '3.28', '3.25']}}
pl
21
20
21
21
{'LiWa': {'equip1': 'Liverpool', 'equip2': 'Watford', 'betclic': ['1,11', '8,50', '17'], 'winamax': ['1,11', '9,50', '21'], 'unibet': ['1.08', '7.35', '15.20']}, 'BuMa': {'equip1': 'Burnley', 'equip2': 'Manchester City', 'betclic': ['11,50', '6,00', '1,20'], 'winamax': ['12,50', '6,50', '1,21'], 'unibet'

In [407]:
# wb = Workbook()  # création du classeur
# ws = wb.active  # création de la feuille
# ws.title = "ldc"  # titre de la feuille
# ws.cell(1,1).value= 'Ligue des Champions'
# sites=[res_betclic,res_winamax,res_unibet]
# name=['Betclic','winamax' ,'unibet']
# r,c=-5,1
# o=-1
# for compet in sites:
#     o+=1
#     r+=7
#     ws.cell(r,c).value = name[o]
#     r+=1
#     row,column=r,c
#     ws.cell(row, column).value = 'Match'
#     ws.cell(row, column+1).value = 'dates'
#     # Merge 3 cells.
#     ws.merge_cells('C'+str(row)+':E'+str(row), 'Merged Range')
#     ws.cell(row,column+2).value = 'cotes'
#     # on set le "curseur"
#     row+=1
#     for i in range(len(res_betclic['ldc']['noms'])):
#         if i%2==0:
#             ws.cell(row, column).value = res_betclic['ldc']['noms'][i] +'-'+ res_betclic['ldc']['noms'][i+1]
#             row += 1  # sur une ligne on change de colonne
    
#     row=r+1
#     column+=1

#     for i in range(len(res_betclic['ldc']['dates'])):
#         # assigne la valeur info à la cellule en ligne(row) et en colonne (column)  
#         ws.cell(row, column).value = res_betclic['ldc']['dates'][i] 
#         row += 1  # sur une ligne on change de colonne
#     row=r+1
#     column+=1

#     for i in range(len(res_betclic['ldc']['cotes'])):
        
#         # assigne la valeur info à la cellule en ligne(row) et en colonne (column)
#         if i%3==0:
#             ws.cell(row, column).value = compet['ldc']['cotes'][i] 
#             column += 1  # sur une ligne on change de colonne
#             ws.cell(row, column).value = compet['ldc']['cotes'][i+1] 
#             column += 1  # sur une ligne on change de colonne
#             ws.cell(row, column).value = compet['ldc']['cotes'][i+2] 
#             column += 1  # sur une ligne on change de colonne
#             row+=1
#         column = 3
        
    
# try:
#     # on sauve l'excel sous la forme xls.
#     #Pour windows
#     # wb.save("C:/Users/HP/OneDrive/Bureau/scrap/paris_sportif"+".xlsx")
#     #Pour Mac
#     wb.save("/Users/thomasbouiniere/Desktop/Scrapping/paris_sportif"+".xlsx")

# except:
#     print('Problème pour sauver l Excel')
browser.quit()
